In [1]:
import gurobipy as gp
from gurobipy import GRB
import json
import random
import numpy as np

# --------------------
# DATA PREPARATION
# --------------------
# Product parameters (fixed costs, gross margins, resource requirements)
fixed_costs = [2, 2, 2, 1, 3, 3, 2, 2, 2, 1, 1, 1, 1, 1]  # $M
gross_margins = [3, 3, 4, 3, 4, 4, 3, 3, 3, 2, 2, 2, 2, 2]  # $M
rd_reqs = [3, 3, 10, 3, 10, 10, 10, 10, 8, 3, 3, 3, 3, 3]  # Units
prod_reqs = [4]*4+[2] * 10  # Units (all 2)

products = [f"Item{i+1}" for i in range(14)]
quarters = list(range(1, 13))
resource_types = ["RD", "Production"]

# Development parameters
dev_types = ["Pkg", "Die"]
dev_costs = {"Pkg": 0.125, "Die": 0.125}  # $M/quarter
dev_duration = 4  # Quarters
last_start = {"Pkg": 8, "Die": 8}  # Last possible start quarter
success_prob = {"Pkg": 0.7, "Die": 0.8}

# Resource parameters
initial_resources = {"RD": 80, "Production": 20}
holding_cost ={"RD": 0, "Production": 0.} # {"RD": 0.2, "Production": 0.15}  # $M/unit/quarter
add_cost = {"RD": 0.2, "Production": 0.1}      # $M/unit
reduce_cost = {"RD": 0.3, "Production": 0.2}    # $M/unit

# --------------------
# SCENARIO GENERATION
# --------------------
def generate_yield_demand_scenarios():
    # Product data structure (as provided)
    product_data = [
        {"id": f"Item{i+1}",
         "yield": [(np.random.uniform(0.75,0.90), 0.3),
                   (np.random.uniform(0.70,0.85), 0.5),
                   (np.random.uniform(0.65,0.80), 0.2)],
         "demand": [(np.random.uniform(1.3,2.5), 0.3),
                    (np.random.uniform(1.0,2.0), 0.5),
                    (np.random.uniform(0.7,1.7), 0.2)]}
        for i in range(14)
    ]

    # Generate 5 yield/demand scenarios
    scenarios = {}
    for i in range(5):
        sc_data = {}
        for product in product_data:
            # Sample yield and demand multipliers
            yield_val = random.choices(
                [val for val, prob in product["yield"]],
                weights=[prob for val, prob in product["yield"]]
            )[0]
            demand_val = random.choices(
                [val for val, prob in product["demand"]],
                weights=[prob for val, prob in product["demand"]]
            )[0]
            sc_data[product["id"]] = {
                "yield": yield_val,
                "demand": demand_val
            }
        scenarios[f"YD_{i+1}"] = sc_data
    return scenarios

# Generate development success scenarios (4 combinations)
def generate_dev_success_scenarios():
    return [
        {"id": "DEV_1", "Pkg_success": 1, "Die_success": 1, "prob": 0.7 * 0.8},
        {"id": "DEV_2", "Pkg_success": 1, "Die_success": 0, "prob": 0.7 * 0.2},
        {"id": "DEV_3", "Pkg_success": 0, "Die_success": 1, "prob": 0.3 * 0.8},
        {"id": "DEV_4", "Pkg_success": 0, "Die_success": 0, "prob": 0.3 * 0.2}
    ]

# Combine into 20 scenarios
yield_demand_scenarios = generate_yield_demand_scenarios()
dev_scenarios = generate_dev_success_scenarios()
combined_scenarios = []

for yd_id, yd_scen in yield_demand_scenarios.items():
    for dev_scen in dev_scenarios:
        sc_id = f"{yd_id}_{dev_scen['id']}"
        prob = (1/5) * dev_scen["prob"]
        combined_scenarios.append({
            "id": sc_id,
            "prob": prob,
            "yield_demand": yd_scen,
            "dev_success": {
                "Pkg": dev_scen["Pkg_success"],
                "Die": dev_scen["Die_success"]
            }
        })

# Precompute availability indicators
def precompute_availability_indicators():
    A = {"Pkg": np.zeros((12, 12)), "Die": np.zeros((12, 12))}  # [quarter, start]

    for d in dev_types:
        for t in range(1, 13):       # Quarter t
            for s in range(1, 13):   # Start quarter s
                if s <= last_start[d] and t >= s + dev_duration:
                    A[d][t-1][s-1] = 1
    return A

availability_indicators = precompute_availability_indicators()

# --------------------
# GUROBI MODEL SETUP
# --------------------
model = gp.Model("Semiconductor_MultiStage_Stochastic_Planning")

# --------------------
# DECISION VARIABLES
# --------------------
# Development start decisions (first-stage)
u_start = {}
for d in dev_types:
    u_start[d] = model.addVars(range(1, last_start[d]+1), vtype=GRB.BINARY,
                              name=f"start_{d}")

# Development investment variables (derived)
D_invest = model.addVars(dev_types, quarters, vtype=GRB.BINARY, name="invest_dev")

# Production decisions (per product, quarter, scenario)
scenario_ids = [sc["id"] for sc in combined_scenarios]  # 创建场景ID列表
x_prod = model.addVars(products, quarters, scenario_ids, vtype=GRB.BINARY, name="produce")
#x_prod = model.addVars(products, quarters, combined_scenarios, vtype=GRB.BINARY,
#                      name="produce")

# Resource variables
y_res = model.addVars(resource_types, quarters, name="resource_level")
h_add = model.addVars(resource_types, quarters, name="resource_add")
f_reduce = model.addVars(resource_types, quarters, name="resource_reduce")

# --------------------
# CONSTRAINTS
# --------------------
# 1. Development scheduling
for d in dev_types:
    # Exactly one start time per development type
    model.addConstr(gp.quicksum(u_start[d][t] for t in range(1, last_start[d]+1)) == 1,
                   f"single_start_{d}")

    # Derive investment variables
    for t in quarters:
        # Sum over possible start quarters that would have active investment at t
        active_terms = []
        for s in range(1, last_start[d]+1):
            if s <= t <= s + dev_duration - 1:
                active_terms.append(u_start[d][s])
        if active_terms:
            model.addConstr(D_invest[d, t] == gp.quicksum(active_terms),
                           f"invest_{d}_{t}")
        else:
            model.addConstr(D_invest[d, t] == 0, f"no_invest_{d}_{t}")

# 2. Production dependencies
for sc in combined_scenarios:
    sc_id = sc["id"]
    # Products 1-4 (require Package)
    for p in products[:4]:
        for t in quarters:
            # Sum over possible start times
            start_terms = []
            for s in range(1, last_start["Pkg"]+1):
                if availability_indicators["Pkg"][t-1, s-1]:
                    start_terms.append(u_start["Pkg"][s])
            if start_terms:
                model.addConstr(x_prod[p, t, sc_id] <= gp.quicksum(start_terms) *
                              sc["dev_success"]["Pkg"],
                              f"pkg_depend_{p}_{t}_{sc_id}")
            else:
                model.addConstr(x_prod[p, t, sc_id] == 0,
                              f"pkg_unavailable_{p}_{t}_{sc_id}")

    # Products 5-8 (require Die)
    for p in products[4:8]:
        for t in quarters:
            start_terms = []
            for s in range(1, last_start["Die"]+1):
                if availability_indicators["Die"][t-1, s-1]:
                    start_terms.append(u_start["Die"][s])
            if start_terms:
                model.addConstr(x_prod[p, t, sc_id] <= gp.quicksum(start_terms) *
                              sc["dev_success"]["Die"],
                              f"die_depend_{p}_{t}_{sc_id}")
            else:
                model.addConstr(x_prod[p, t, sc_id] == 0,
                              f"die_unavailable_{p}_{t}_{sc_id}")

# 3. Resource constraints
for sc in combined_scenarios:
    sc_id = sc["id"]
    for t in quarters:
        # R&D constraint (production + development)
        rd_usage = gp.quicksum(
            rd_reqs[i] * x_prod[p, t, sc_id] for i, p in enumerate(products)
        )
        rd_usage += D_invest["Pkg", t] + 2 * D_invest["Die", t]
        model.addConstr(rd_usage <= y_res["RD", t], f"RD_capacity_{t}_{sc_id}")

        # Production line constraint
        prod_usage = gp.quicksum(
            prod_reqs[i] * x_prod[p, t, sc_id] for i, p in enumerate(products)
        )
        model.addConstr(prod_usage <= y_res["Production", t],
                      f"production_capacity_{t}_{sc_id}")

# 4. Resource dynamics
for r in resource_types:
    # Initial resource level
    model.addConstr(y_res[r, 1] == initial_resources[r], f"init_{r}")

    # No adjustments in last quarter
    for t in range(1, 13):
        model.addConstr(h_add[r, t] == 0, f"no_add_{r}_Q{t}")
        model.addConstr(f_reduce[r, t] == 0, f"no_reduce_{r}_Q{t}")

    # Quarter-to-quarter transitions
    for t in range(2, 13):
        model.addConstr(
            y_res[r, t] == y_res[r, t-1] + h_add[r, t-1] - f_reduce[r, t-1],
            f"resource_flow_{r}_{t}"
        )

# 5. Non-anticipation constraints
# Q1 decisions must be identical across all scenarios
for p in products:
    q1_vars = [x_prod[p, 1, sc["id"]] for sc in combined_scenarios]
    for i in range(1, len(q1_vars)):
        model.addConstr(q1_vars[0] == q1_vars[i], f"non_ant_{p}_Q1")

# --------------------
# OBJECTIVE FUNCTION
# --------------------
# Expected profit from production
production_profit = 0
for sc in combined_scenarios:
    sc_id = sc["id"]
    sc_profit = 0
    for i, p in enumerate(products):
        for t in quarters:
            # Get scenario multipliers
            yd = sc["yield_demand"][p]
            revenue = gross_margins[i] * yd["yield"] * yd["demand"]
            cost = fixed_costs[i]
            sc_profit += (revenue - cost) * x_prod[p, t, sc_id]
    production_profit += sc["prob"] * sc_profit

# Resource costs
resource_costs = 0
for r in resource_types:
    for t in quarters:
        resource_costs += holding_cost[r] * y_res[r, t]
        resource_costs += add_cost[r] * h_add[r, t]
        resource_costs += reduce_cost[r] * f_reduce[r, t]

# Development costs
dev_costs_total = 0
for d in dev_types:
    for t in quarters:
        dev_costs_total += dev_costs[d] * D_invest[d, t]

# Set objective
total_profit = production_profit - resource_costs - dev_costs_total
model.setObjective(total_profit, GRB.MAXIMIZE)

# --------------------
# SOLVE & OUTPUT
# --------------------
model.Params.NonConvex = 2  # Handle bilinear terms in availability constraints
model.optimize()

# Output results
if model.status == GRB.OPTIMAL:
    print(f"\nOptimal Expected Profit: ${model.ObjVal:,.2f}M")

    # Print development decisions
    print("\nDevelopment Start Decisions:")
    for d in dev_types:
        for t in range(1, last_start[d]+1):
            if u_start[d][t].X > 0.5:
                print(f"- {d} development starts in Q{t}")

    # Print resource adjustments
    print("\nResource Adjustments (Q1-Q11):")
    for r in resource_types:
        print(f"{r}:")
        for t in range(1, 12):  # Q1-Q11 only
            add_val = h_add[r, t].X
            reduce_val = f_reduce[r, t].X
            if add_val > 0 or reduce_val > 0:
                print(f"  Q{t}: Add {add_val:.1f}, Reduce {reduce_val:.1f}")

    # Print production summary
    print("\nProduction Summary (Selected in Scenarios):")
    for p in products:
        total_scenarios = len(combined_scenarios)
        selected_in = sum(1 for sc in combined_scenarios
                        if any(x_prod[p, t, sc["id"]].X > 0.5 for t in quarters))
        print(f"- {p}: {selected_in}/{total_scenarios} scenarios")
else:
    print("No optimal solution found. Status:", model.status)

# Save model for inspection
model.write("semiconductor_plan.lp")

Restricted license - for non-production use only - expires 2026-11-23
Set parameter NonConvex to value 2
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Non-default parameters:
NonConvex  2



GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information